In [1]:

import pandas as pd
original_dataset = pd.read_csv("../all_pairs - old.tsv", sep='\t') 

annotations_df = pd.read_csv("extracted_reconciled_annotations.tsv", sep='\t') 

original_dois = original_dataset['doi'].tolist()
covid_preprint = original_dataset["covid_preprint"].tolist()
covid_preprint = {original_dois[x]:covid_preprint[x] for x in range(len(original_dois))}


In [2]:
max_id = max([int(x) for x in set(annotations_df["abstract_id"])])
print (max_id)

185


In [3]:
annotations_df = annotations_df[annotations_df['score'].notna()]

dois = open("dois.txt","r").read().strip().split("\n")
dois = {x+1:dois[x] for x in range(len(dois))}

print (len(dois))
dois[184]

184


'10.1101/869313'

In [4]:
 normalise_sections = {'conclusion':"conclusions",
 'conclusion':"conclusions",
 'conclusion  ':"conclusions",
 'conclusions':"conclusions",
 'concusions':"conclusions",
 'contex':"context",
 'context':"context",
 'context  ':"context",
 'context  removed':"context",
 'countext':"context",
 'new':"new",
 'reslts':'results',
 'resuls':'results',
 'result':'results',
 'results':'results',
 'results/context':'results/context',
 'resutls':'results',
 'resuts':'results'}

normalise_labels = {'nounchange': 'nounchange', 'effect-': 'effect-', 'stat-': 'stat-', 'added': 'added', 'removed': 'removed', 'stat+': 'stat+', 'stat-1': 'stat-', 'effect+': 'effect+', 'statinfo': 'stat', 'effectreverse': 'effectreverse', 'aded': 'added', 'effectt+': 'effect+', 'nouchange': 'nounchange', 'effect': 'effect', 'results': 'results', 'nounchage': 'nounchange', 'stat': 'stat', 'ef': 'effect', 'added  ': 'added', 'emoved': 'removed', 'remove': 'removed'}

In [5]:
out_ann = open("final_reconciled_annotations.csv","w")
out_ann.write("page_id,doi,covid_preprint,annotator,section,label,label+modifier,label_modifier,score,score+modifier,score_modifier\n")

final_columns = {}
for id in range(1,185):
    overall_score = {x:{p:[] for z,p in normalise_labels.items()} for y,x in normalise_sections.items()}
    scores = annotations_df.loc[annotations_df['abstract_id'] == id]["score"].tolist()
    annotators = annotations_df.loc[annotations_df['abstract_id'] == id]["annotator"].tolist()
    scores = [[annotators[x],scores[x]] for x in range(len(scores)) if "_" in scores[x]]
    doi = dois[id]
    preprint = covid_preprint[doi]
    for el in scores:
            ann = el[0].replace(","," ")
            score = el[1]
            score = score.replace(" ","")
            score = score.strip()

            if len(score.split("_"))>2 and score.split("_")[0] in normalise_sections:
                section = normalise_sections[score.split("_")[0]]
                label = normalise_labels[score.split("_")[1]]
                
                # check if there is a modifier in the label
                last_char = label[-1]
                if last_char=="+" or last_char=="-":
                    label_modifier = last_char
                    label_without_modifier = label.replace("+","").replace("-","")
                else:
                    label_modifier=""
                    label_without_modifier = label

                sc = score.split("_")[-1].strip()

                score_modifier = [x for x in sc if x == "+" or x == "-"]
                if len(score_modifier)>0:
                    score_modifier = score_modifier[0]
                    sc_without_modifier = sc.replace(score_modifier,"")
                else:
                    score_modifier = ""
                    sc_without_modifier = sc

                try:
                    int_sc = int(''.join(c for c in sc if c.isdigit()))
                # this happens if the annotator forgot to assign a value, we add 1 here
                except ValueError:
                    sc = ""
            
                
                out_ann.write(str(id)+","+doi+","+str(preprint)+","+ann+","+section+","+label_without_modifier+","+label+","+label_modifier+","+sc_without_modifier+","+sc+","+score_modifier+"\n")

out_ann.close()